In [14]:
from utils import *
import numpy as np
import pickle
import time

In [3]:
t,v = get_data()

In [4]:
ui_mtx = create_ui_matrix(t)

In [13]:
ui_mtx.shape

(6040, 3224)

In [11]:
itzik = ui_mtx[:3,:3]
itzik

array([[4, 5, 5],
       [0, 0, 0],
       [0, 0, 0]])

In [14]:
itzik.mean(axis=1)

array([4.66666667, 0.        , 0.        ])

In [10]:
rel_cols = ui_mtx[:,[6,1]]
rel_cols = rel_cols[(rel_cols!=0).all(axis=1)]
rel_cols

array([[4, 5],
       [5, 3],
       [4, 3]])

In [11]:
c = rel_cols[:,0] - rel_cols[:,1]
c.mean()

0.6666666666666666

In [12]:
t[t[:,2] == 0]

array([], shape=(0, 3), dtype=int32)

In [ ]:
test_dic = {'a':1,'b':2,'c':3}
with open('learned_paramaters/dic_test.pickle', 'wb') as f:
    pickle.dump(test_dic, f)

In [ ]:
with open('learned_paramaters/dic_test.pickle', 'rb') as f:
    moti = pickle.load(f)

In [ ]:
moti

In [ ]:
with open('learned_paramaters/popularity_differences_params.pickle', 'rb') as f:
    moti = pickle.load(f)

In [ ]:
len(moti.keys())

In [ ]:
3224*3223

In [17]:
def func(row):
    print(row[0]*4)
np.apply_along_axis(func,1,itzik)

16
0
0


array([None, None, None], dtype=object)

In [2]:
np.random.normal(0, 0.1, (4, 5))

array([[ 0.12577747,  0.13286638, -0.09803337,  0.0981204 ,  0.22344792],
       [-0.03353047,  0.0374203 , -0.00774108, -0.14836026,  0.11435774],
       [ 0.08359585, -0.10718975, -0.00746879, -0.08252795, -0.18735923],
       [-0.06051676, -0.05658918,  0.06794939, -0.14875344, -0.13690487]])

In [19]:
ui_mtx.mean(axis=1).shape

(6040,)

In [27]:
# ui matrix but all zero values are converted to nan, for mean calculation
rating_mtx = ui_mtx.copy().astype(float)
rating_mtx[rating_mtx == 0] = np.nan
# calculate mean but ignore nans
np.nanmean(rating_mtx, axis=1)


array([3.41772152, 3.86750789, 4.23142857, ..., 3.85714286, 2.55      ,
       4.10526316])